In [661]:
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt
from typing import Tuple

# Project Idea: Find the (Non-Complex) Eigenvalue of a Matrix

It's from scratch-ish with some use of `numpy` because I don't want to write a hundred for-loops.

## Power Method

Source: [this link](https://www.youtube.com/watch?v=TS32rBteHh4)

Finds the eigenvalue with the highest magnitude.

In [662]:
def power_eig(A:np.ndarray, k:int=100) -> Tuple[float, np.ndarray]:
	size, _ = A.shape
	vec = np.ones(size)

	A = A.astype(float)
	vec = vec.astype(float)

	for i in range(k):
		vec = np.matmul(A, vec)
		coeff = max(vec)
		vec = vec / coeff

	last = np.matmul(A, vec)
	return np.average(last/vec), vec

In [663]:
test_matrix = np.array([[5., 8., 16.],
						[4., 1., 8.],
						[-4., -4., -11.]])
max_eival, max_eivec = power_eig(test_matrix)
print(max_eival, max_eivec)

-3.0 [-1.4 -0.6  1. ]


In [664]:
max_eivec, np.matmul(test_matrix, max_eivec) # A*v = -3*v

(array([-1.4, -0.6,  1. ]), array([ 4.2,  1.8, -3. ]))

## Shifted Inverse Power Method

Finding eigenvalues that isn't the biggest one

In [ ]:
def shifted_ip_eig(A:np.ndarray, k:int=100):
	...